# [MARKETING ANALYTICS] DATA PREPARATION


**OBJETIVO DO PROJETO:**

O principal objetivo deste projeto é aumentar a eficácia das campanhas de marketing do banco.

**PREMISSAS:**

1. A análise considera apenas pessoas físicas, excluindo contas corporativas ou jurídicas.
2. Apenas clientes contactados durante a campanha mais recente são incluídos na análise, excluindo dados de campanhas anteriores.
3. Consideram-se apenas os contatos feitos via telefone celular e telefone fixo, excluindo outros meios de contato.


**FONTES INFORMACIONAIS:**

1. Dados das campanhas de marketing direto realizadas pelo banco, disponíveis no Bank Marketing Dataset da UCI Machine Learning Repository.
2. Indicadores econômicos, como taxa de variação de emprego, índice de preços ao consumidor, índice de confiança do consumidor e taxa Euribor de 3 meses.



**OBJETIVO(S) DO SCRIPT:**

O trabalho está seguindo a metodologia CRISP-DM e este script corresponde a fase de Data Preparation.

1. Carregamento dos dados brutos.
1. Separar os grupos entre controle, teste e validação.
1. Analisar e tratar outliers.
1. Tratar dados ausentes.
1. Criar variáveis derivadas.
1. Codificar variáveis.
1. Filtrar variáveis.
1. Criar registro analítico para modelagem.






## 1. BIBLIOTECAS E CONFIGURAÇÕES



### 1.1 BIBLIOTECAS

In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split

### 1.2 CONFIGURAÇÕES

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
warnings.filterwarnings('ignore')

### 1.3 FUNÇÕES

In [3]:
def exibir_valores_ausentes(df):
    """
    Exibe as colunas de um DataFrame que possuem valores ausentes.

    A função calcula a quantidade absoluta e a porcentagem de valores ausentes
    em cada coluna do DataFrame fornecido e exibe apenas as colunas que têm valores ausentes.
    A exibição é ordenada pela porcentagem de valores ausentes em ordem decrescente,
    com a porcentagem formatada para duas casas decimais.

    Parâmetros:
    df (pandas.DataFrame): O DataFrame a ser analisado.

    Retorna:
    None: Esta função não retorna nada. Ela imprime a quantidade e a porcentagem de valores ausentes.
    """
    # Calcular a quantidade absoluta de valores ausentes em cada coluna
    valores_ausentes = df.isna().sum()

    # Calcular a porcentagem de valores ausentes em cada coluna
    porcentagem_ausentes = (valores_ausentes / len(df)) * 100

    # Criar um DataFrame combinando a quantidade absoluta e a porcentagem de valores ausentes
    valores_ausentes_df = pd.DataFrame({
        'Quantidade Ausente': valores_ausentes,
        'Porcentagem Ausente (%)': porcentagem_ausentes
    })

    # Filtrar o DataFrame para mostrar apenas as colunas com valores ausentes
    valores_ausentes_df = valores_ausentes_df[valores_ausentes_df['Quantidade Ausente'] > 0]

    # Ordenar pela coluna de porcentagem ausente em ordem decrescente
    valores_ausentes_df = valores_ausentes_df.sort_values(by='Porcentagem Ausente (%)', ascending=False)

    # Formatando a porcentagem para mostrar apenas duas casas decimais
    valores_ausentes_df['Porcentagem Ausente (%)'] = valores_ausentes_df['Porcentagem Ausente (%)'].apply(lambda x: f"{x:.2f}")

    # Exibir o DataFrame resultante
    display(valores_ausentes_df)

In [4]:
def categorize_generation(age):
    """
    Categoriza a geração com base na idade do usuário em 2010.

    Parâmetros:
    age (int): Idade do usuário em 2010.

    Retorna:
    str: Geração correspondente à idade fornecida.
    """
    if age <= 13:
        return 'Gen_Z'
    elif 14 <= age <= 29:
        return 'Gen_Y'
    elif 30 <= age <= 45:
        return 'Gen_X'
    elif 46 <= age <= 64:
        return 'Gen_Boomer'
    else:
        return 'Gen_Silent'

In [5]:
def analisar_outliers_iqr(df):
    """
    Faz a análise de outliers pelo método IQR para todas as variáveis numéricas do DataFrame.

    Args:
        df (pd.DataFrame): DataFrame com as variáveis a serem analisadas.

    Returns:
        pd.DataFrame: DataFrame com os limites inferior e superior, quantidade de outliers, percentil 99,
                      quantidade de casos acima do percentil 99, valor mínimo e valor máximo de cada variável.
    """
    # Selecionar apenas as colunas numéricas
    num_df = df.select_dtypes(include=['number'])

    # Lista para armazenar os resultados
    resultados = []

    for coluna in num_df.columns:
        # Calcular Q1, Q3, IQR
        Q1 = num_df[coluna].quantile(0.25)
        Q3 = num_df[coluna].quantile(0.75)
        IQR = Q3 - Q1

        # Calcular limites inferior e superior
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR

        # Contar casos abaixo do limite inferior e acima do limite superior
        qtd_abaixo_inferior = (num_df[coluna] < limite_inferior).sum()
        qtd_acima_superior = (num_df[coluna] > limite_superior).sum()

        # Calcular percentil 99
        percentil_99 = num_df[coluna].quantile(0.99)

        # Contar casos acima do percentil 99
        qtd_acima_percentil_99 = (num_df[coluna] > percentil_99).sum()

        # Calcular valor mínimo e máximo
        valor_minimo = num_df[coluna].min()
        valor_maximo = num_df[coluna].max()

        # Adicionar os resultados à lista
        resultados.append({
            'Variável': coluna,
            'Limite Inferior': round(limite_inferior, 2),
            'Limite Superior': round(limite_superior, 2),
            'Qtd. Abaixo do Limite Inferior': qtd_abaixo_inferior,
            'Qtd. Acima do Limite Superior': qtd_acima_superior,
            'Percentil 99': round(percentil_99, 2),
            'Qtd. Acima do Percentil 99': qtd_acima_percentil_99,
            'Valor Mínimo': round(valor_minimo, 2) if isinstance(valor_minimo, float) else valor_minimo,
            'Valor Máximo': round(valor_maximo, 2) if isinstance(valor_maximo, float) else valor_maximo
        })

    # Converter a lista de resultados para DataFrame
    resultados_df = pd.DataFrame(resultados)

    # Ajustar configurações de exibição do Pandas
    pd.set_option('display.float_format', lambda x: '%.2f' % x)

    return resultados_df

In [6]:
def fill_missing_with_mode(group):
    """
    Preenche os valores ausentes em um grupo com a moda do grupo.

    Parameters:
    group (pd.Series): Série de dados agrupados.

    Returns:
    pd.Series: Série com valores ausentes preenchidos pela moda do grupo.
    """
    mode_value = group.mode().iloc[0]
    return group.fillna(mode_value)

In [7]:
def fill_missing_with_train_mode(row, modes):
    """
    Preenche o valor ausente da variável 'education' em uma linha de DataFrame
    com a moda correspondente do DataFrame de treinamento.

    Parameters:
    row (pd.Series): Linha do DataFrame.
    modes (pd.Series): Série com as modas de 'education' para cada 'job' calculadas em df_train.

    Returns:
    any: Valor de 'education' preenchido ou original se não estiver ausente.
    """
    if pd.isnull(row['education']):
        return modes.get(row['job'], row['education'])
    return row['education']

## 2. LEITURA DOS DADOS





### 2.1 OBTENÇÃO DOS DADOS

In [8]:
df = pd.read_csv('https://raw.githubusercontent.com/manuelcalcada/marketing-analytics/main/1.%20dataset/bank-additional-full.csv', sep = ';', low_memory=False, na_values=['unknown', 999, -1, 'nonexistent'])
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261.0,1,NaN,0,NaN,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149.0,1,NaN,0,NaN,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226.0,1,NaN,0,NaN,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151.0,1,NaN,0,NaN,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307.0,1,NaN,0,NaN,1.1,93.994,-36.4,4.857,5191.0,no


### 2.2 FILTROS

Removendo variáveis multicolineares identificadas na fase anterior.

In [9]:
df = df.drop(columns=['duration', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'nr.employed'])

### 2.3 PARTIÇÕES DE TREINO, TESTE E VALIDAÇÃO

A divisão dos dados entre treino, teste e validação out of time assegura que o modelo seja treinado, testado e validado em diferentes períodos temporais, melhorando sua capacidade de generalização e desempenho em dados futuros.

In [10]:
#proporção por mês de campanha
df.month.value_counts(normalize=True)

month
may    0.334296
jul    0.174177
aug    0.149995
jun    0.129115
nov    0.099568
apr    0.063902
oct    0.017432
sep    0.013839
mar    0.013256
dec    0.004419
Name: proportion, dtype: float64

Para a partição de validação out of time usaremos os dados dos meses de setembro e outubro, que corresponde a mais ou menos 3% da base inicial.

In [11]:
#validação
df_val = df[df['month'].isin(['oct', 'nov'])]
df_val.shape

(4819, 16)

In [12]:
#treino e teste
df_train, df_test = train_test_split(df[~df['month'].isin(['oct', 'nov'])], test_size=0.3, stratify=df[~df['month'].isin(['oct', 'nov'])].y, random_state=42)
print(df_train.shape)
print(df_test.shape)

(25458, 16)
(10911, 16)


In [13]:
proporcao_val = df_val['y'].value_counts(normalize=True).to_frame().reset_index()
proporcao_val.columns = ['Categoria', 'Proporcao_val']

proporcao_train = df_train['y'].value_counts(normalize=True).to_frame().reset_index()
proporcao_train.columns = ['Categoria', 'Proporcao_train']

proporcao_test = df_test['y'].value_counts(normalize=True).to_frame().reset_index()
proporcao_test.columns = ['Categoria', 'Proporcao_test']

proporcoes_combinadas = proporcao_val.merge(proporcao_train, on='Categoria', how='outer')
proporcoes_combinadas = proporcoes_combinadas.merge(proporcao_test, on='Categoria', how='outer')

proporcoes_combinadas = proporcoes_combinadas.fillna(0)

display(proporcoes_combinadas)

,Categoria,Proporcao_val,Proporcao_train,Proporcao_test
0,no,0.848309,0.892529,0.892494
1,yes,0.151691,0.107471,0.107506


Conforme solicitado, a proporção de conversões de treino e teste são iguais (11%). Para a validação, esse número é ligeiramente maior (15%), mas ainda assim representa um cenário típico que o modelo enfrentaria em produção.

## 3. ANÁLISE E TRATAMENTO DE OUTLIERS

O tratamento de outliers é crucial em projetos de análise de dados porque esses valores extremos podem distorcer significativamente os resultados e as conclusões. Outliers podem influenciar a média, ampliar a variância e afetar a performance de modelos preditivos. Identificá-los e tratá-los adequadamente melhora a qualidade dos dados, levando a análises mais precisas e modelos mais robustos. Isso garante que as decisões baseadas nos dados sejam fundamentadas e confiáveis, evitando interpretações errôneas e potenciais prejuízos.

In [14]:
display(analisar_outliers_iqr(df_train))

,Variável,Limite Inferior,Limite Superior,Qtd. Abaixo do Limite Inferior,Qtd. Acima do Limite Superior,Percentil 99,Qtd. Acima do Percentil 99,Valor Mínimo,Valor Máximo
0,age,9.50,69.50,0,231,68.00,249,17.00,95.00
1,campaign,-2.00,6.00,0,1635,16.00,221,1.00,56.00
2,pdays,-4.50,15.50,0,20,18.00,7,0.00,26.00
3,previous,0.00,0.00,0,2998,2.00,150,0.00,6.00
4,euribor3m,-4.08,10.39,0,0,4.97,126,0.63,4.97


A tabela anterior demonstra os limites superiores e inferiores das variáveis numéricas utilizando o método da amplitude interquantílica (IQR), assim como a quantidade de registros que ficam fora desses limites, o percentil 99 e a quantidade de registros acima desse percentil 99.

As variáveis parecem estar com valores que realmente poderiam ter, porém para melhorar a sensibilidade na modelagem faremos alguns tratamentos conforme a seguir:

- age: nenhum
- campaign: P99
- pdays: P99
- previous: nenhum
- euribor3m: nenhum

In [15]:
df_train['campaign'] = np.where(df_train['campaign'] > 16, 16, df_train['campaign'])
df_train['pdays'] = np.where(df_train['pdays'] > 18, 18, df_train['pdays'])

df_test['campaign'] = np.where(df_test['campaign'] > 16, 16, df_test['campaign'])
df_test['pdays'] = np.where(df_test['pdays'] > 18, 18, df_test['pdays'])

df_val['campaign'] = np.where(df_val['campaign'] > 16, 16, df_val['campaign'])
df_val['pdays'] = np.where(df_val['pdays'] > 18, 18, df_val['pdays'])

## 4. ANÁLISE E TRATAMENTO DE VALORES AUSENTES

Tratar dados ausentes é crucial, pois os modelos preditivos que iremos aplicar necessitam de dados preenchidos para garantir precisão e robustez nos resultados. Podemos fazer esse tratamento criando novas categorias ou assimilando valores pré-existentes ou não.


In [16]:
#verificando a quantidade de dados ausentes
exibir_valores_ausentes(df)

,Quantidade Ausente,Porcentagem Ausente (%)
pdays,39673,96.32
poutcome,35563,86.34
default,8597,20.87
education,1731,4.20
housing,990,2.40
loan,990,2.40
job,330,0.80
marital,80,0.19


Na análise exploratória, identificamos uma correlação entre as variáveis de educação e emprego. Portanto, ao tratar a variável de educação, é essencial seguir uma lógica que relacione profissões com níveis educacionais, evitando assim discrepâncias nos dados. Uma abordagem viável é preencher os valores ausentes de educação utilizando a moda dos valores de educação para cada tipo de emprego.

In [17]:
#moda dos dados categóricos
df_train[['education', 'job', 'marital', 'loan','housing','default']].mode().T

,0
education,university.degree
job,admin.
marital,married
loan,no
housing,yes
default,no


In [18]:
#mediana de pdays
df.pdays.median()

6.0

In [19]:
#moda de educação por emprego
display(df_train.groupby('job')['education'].agg(pd.Series.mode))

job
admin.             university.degree
blue-collar                 basic.9y
entrepreneur       university.degree
housemaid                   basic.4y
management         university.degree
retired                     basic.4y
self-employed      university.degree
services                 high.school
student                  high.school
technician       professional.course
unemployed         university.degree
Name: education, dtype: object

In [20]:
#preenchendo dados ausentes de educação
modes = df_train.groupby('job')['education'].agg(lambda x: x.mode().iloc[0])
df_train['education'] = df_train.groupby('job')['education'].transform(fill_missing_with_mode)
df_test['education'] = df_test.apply(lambda row: fill_missing_with_train_mode(row, modes), axis=1)
df_val['education'] = df_val.apply(lambda row: fill_missing_with_train_mode(row, modes), axis=1)

In [21]:
df_train[df_train['job'].isnull()]['education'].value_counts(normalize=True, dropna=False)

education
NaN   1.00
Name: proportion, dtype: float64

In [22]:
df_train[df_train['education'].isnull()]['job'].value_counts(normalize=True, dropna=False)

job
NaN   1.00
Name: proportion, dtype: float64

Todos os valores ausentes em educação estão relacionados a valores ausentes em emprego, e vice-versa. Esses valores serão tratados usando a moda das variáveis correspondentes. A variável "Marital" também será tratada utilizando sua moda.

Assumiremos que os valores ausentes em "loan" e "housing" indicam que o cliente não possui esses serviços, portanto, serão preenchidos com "no".

A variável "default" apresenta um comportamento distinto para dados ausentes em comparação aos clientes sem inadimplência. Assim, consideraremos que os valores ausentes em "default" indicam que o cliente possui inadimplência.

A variável "poutcome" só é preenchida quando o cliente foi contatado em uma campanha anterior, portanto, os valores ausentes serão preenchidos com "NA". Da mesma forma, "pdays" será preenchido com 0, indicando que o cliente não foi contatado anteriormente. Contudo, não podem existir "pdays" igual a 0 para "poutcome" não ausente, de lógica parecida a emprego-educação anterior, por isso iremos utilizar o valor da mediana de "pdays" para quando "poutcome" estiver preenchido.

In [23]:
df_train['education'] = df_train['education'].fillna(df_train['education'].mode()[0])
df_test['education'] = df_test['education'].fillna(df_train['education'].mode()[0])
df_val['education'] = df_val['education'].fillna(df_train['education'].mode()[0])

df_train['job'] = df_train['job'].fillna(df_train['job'].mode()[0])
df_test['job'] = df_test['job'].fillna(df_train['job'].mode()[0])
df_val['job'] = df_val['job'].fillna(df_train['job'].mode()[0])

df_train['marital'] = df_train['marital'].fillna(df_train['marital'].mode()[0])
df_test['marital'] = df_test['marital'].fillna(df_train['marital'].mode()[0])
df_val['marital'] = df_val['marital'].fillna(df_train['marital'].mode()[0])

df_train['loan'] = df_train['loan'].fillna('no')
df_test['loan'] = df_test['loan'].fillna('no')
df_val['loan'] = df_val['loan'].fillna('no')

df_train['housing'] = df_train['housing'].fillna('no')
df_test['housing'] = df_test['housing'].fillna('no')
df_val['housing'] = df_val['housing'].fillna('no')

df_train['default'] = df_train['default'].fillna('yes')
df_test['default'] = df_test['default'].fillna('yes')
df_val['default'] = df_val['default'].fillna('yes')

df_train['poutcome'] = df_train['poutcome'].fillna('NA')
df_test['poutcome'] = df_test['poutcome'].fillna('NA')
df_val['poutcome'] = df_val['poutcome'].fillna('NA')

df_train[~df_train['poutcome'].isnull()]['pdays'] = 6
df_test[~df_test['poutcome'].isnull()]['pdays'] = 6
df_val[~df_val['poutcome'].isnull()]['pdays'] = 6

df_train['pdays'] = df_train['pdays'].fillna(0)
df_test['pdays'] = df_test['pdays'].fillna(0)
df_val['pdays'] = df_val['pdays'].fillna(0)

In [24]:
#verificando a quantidade de dados ausentes após tratamentos
exibir_valores_ausentes(df_train)
exibir_valores_ausentes(df_test)
exibir_valores_ausentes(df_val)

,Quantidade Ausente,Porcentagem Ausente (%)


,Quantidade Ausente,Porcentagem Ausente (%)


,Quantidade Ausente,Porcentagem Ausente (%)


## 5. VARIÁVEIS DERIVADAS

A criação de variáveis derivadas envolve transformar dados brutos em novas características que podem revelar padrões ocultos e melhorar a performance dos modelos preditivos, enriquecendo a análise e a interpretação dos dados.

Neste projeto, identificamos a possibilidade de construção de 3 variáveis:
1. Geração
1. Categoria do Trabalho
1. Categoria de Escolaridade

### 5.1 GERAÇÃO

In [25]:
df_train['generation'] = df_train['age'].apply(categorize_generation)
df_test['generation'] = df_test['age'].apply(categorize_generation)
df_val['generation'] = df_val['age'].apply(categorize_generation)

df_train['generation'].value_counts()

generation
Gen_X         14614
Gen_Boomer     6854
Gen_Y          3659
Gen_Silent      331
Name: count, dtype: int64

### 5.2 CATEGORIA DO TRABALHO

In [26]:
job_categories = {
    'admin.': 'Profissionais de Escritório',
    'management': 'Profissionais de Escritório',
    'technician': 'Profissionais de Escritório',
    'blue-collar': 'Trabalhadores Manuais',
    'services': 'Trabalhadores Manuais',
    'housemaid': 'Trabalhadores Manuais',
    'self-employed': 'Autônomos',
    'entrepreneur': 'Autônomos',
    'unemployed': 'Não Empregados',
    'retired': 'Não Empregados',
    'student': 'Não Empregados'
}

df_train['job_category'] = df_train['job'].map(job_categories)
df_test['job_category'] = df_test['job'].map(job_categories)
df_val['job_category'] = df_val['job'].map(job_categories)

df_train['job_category'].value_counts()

job_category
Profissionais de Escritório    12422
Trabalhadores Manuais           9292
Não Empregados                  2107
Autônomos                       1637
Name: count, dtype: int64

### 5.3 CATEGORIA DE ESCOLARIDADE

In [27]:
education_categories = {
    'university.degree': 'Ensino Superior',
    'high.school': 'Ensino Médio',
    'basic.4y': 'Educação Básica',
    'basic.6y': 'Educação Básica',
    'basic.9y': 'Educação Básica',
    'illiterate': 'Educação Básica',
    'professional.course': 'Profissionalizante'
}

df_train['education_category'] = df_train['education'].map(education_categories)
df_test['education_category'] = df_test['education'].map(education_categories)
df_val['education_category'] = df_val['education'].map(education_categories)

df_train['education_category'].value_counts()

education_category
Educação Básica       8333
Ensino Superior       7688
Ensino Médio          6078
Profissionalizante    3359
Name: count, dtype: int64

## 6. CODIFICAÇÃO DE VARIÁVEIS

A codificação de variáveis é crucial em modelos de Machine Learning, convertendo dados categóricos em formatos numéricos adequados. Técnicas como one-hot encoding e label encoding garantem que o modelo interprete corretamente essas variáveis, melhorando a precisão e a eficiência das análises.

Atributos ordinais que serão tratados com ordinal encoding:
- education
- education_category
- generation

Atributos não ordinais que serão tratados com dummy encoding:
- job
- job_category
- marital
- contact
- poutcome

### 6.1 DUMMY ENCODING

In [28]:
df_train['default'] = df_train['default'].map({'yes': 1, 'no': 0})
df_test['default'] = df_test['default'].map({'yes': 1, 'no': 0})
df_val['default'] = df_val['default'].map({'yes': 1, 'no': 0})

df_train['housing'] = df_train['housing'].map({'yes': 1, 'no': 0})
df_test['housing'] = df_test['housing'].map({'yes': 1, 'no': 0})
df_val['housing'] = df_val['housing'].map({'yes': 1, 'no': 0})

df_train['loan'] = df_train['loan'].map({'yes': 1, 'no': 0})
df_test['loan'] = df_test['loan'].map({'yes': 1, 'no': 0})
df_val['loan'] = df_val['loan'].map({'yes': 1, 'no': 0})

df_train['y'] = df_train['y'].map({'yes': 1, 'no': 0})
df_test['y'] = df_test['y'].map({'yes': 1, 'no': 0})
df_val['y'] = df_val['y'].map({'yes': 1, 'no': 0})

In [29]:
df_train = pd.get_dummies(df_train, columns=["job", "job_category", "marital", "contact", "poutcome"], drop_first=False)
df_test = pd.get_dummies(df_test, columns=["job", "job_category", "marital", "contact", "poutcome"], drop_first=False)
df_val = pd.get_dummies(df_val, columns=["job", "job_category", "marital", "contact", "poutcome"], drop_first=False)

### 6.2 ORDINAL ENCODING

In [30]:
education_mapping = {
    'illiterate': 0,
    'basic.4y': 1,
    'basic.6y': 2,
    'basic.9y': 3,
    'high.school': 4,
    'professional.course': 5,
    'university.degree': 6,
    np.nan: -1
}

df_train['education'] = df_train['education'].map(education_mapping)
df_test['education'] = df_test['education'].map(education_mapping)
df_val['education'] = df_val['education'].map(education_mapping)

In [31]:
education_category_mapping = {
    'Educação Básica': 0,
    'Ensino Médio': 1,
    'Profissionalizante': 2,
    'Ensino Superior': 3,
    np.nan: -1
}

df_train['education_category'] = df_train['education_category'].map(education_category_mapping)
df_test['education_category'] = df_test['education_category'].map(education_category_mapping)
df_val['education_category'] = df_val['education_category'].map(education_category_mapping)

In [32]:
generation_mapping = {
    'Gen_Silent': 0,
    'Gen_Boomer': 1,
    'Gen_X': 2,
    'Gen_Y': 3,
    np.nan: -1
}

df_train['generation'] = df_train['generation'].map(generation_mapping)
df_test['generation'] = df_test['generation'].map(generation_mapping)
df_val['generation'] = df_val['generation'].map(generation_mapping)

As variáveis referentes ao mês da campanha e ao dia da semana em que o cliente foi contatado não serão incluídas na modelagem. Esses dados não representam características do cliente, mas sim da operação da campanha. Portanto, enquanto são úteis para otimizar as estratégias de campanha, não contribuem diretamente para a definição do perfil do público-alvo. Por esse motivo, optamos por não aplicar encoding nessas variáveis.

## GERAÇÃO DE REGISTRO ANALÍTICO CONSOLIDADO

In [33]:
df_train.to_csv('df_train.csv', index=False)
df_test.to_csv('df_test.csv', index=False)
df_val.to_csv('df_val.csv', index=False)

Após o tratamento, os dados estão agora prontos para a modelagem. Alguns modelos podem requerer normalização, redução de dimensionalidade e balanceamento de dados, etapas que serão realizadas diretamente durante o processo de modelagem.